In [ ]:
#| default_exp nostr

In [ ]:
#| hide
from nbdev.showdoc import *

# Nostr Core

> Learn about the nostr protocol and how we call it from python with `python-nostr`

>This repository relies heavily on [python-nostr](https://github.com/jeffthibault/python-nostr) for underlying functionality. This notebook will walk you through the key characteristics of the nostr protocol and also make some changes to the classes from `python-nostr` that will be used in the basic client implementation.

## What is nostr?
This notebook will attempt to describe nostr via python code. There are many resources dedicated to describing nostr in words. See:

 - [the readme on the nostr protocol github](https://github.com/nostr-protocol/nostr#readme)
 - [the Nostr Improvement Proposal (NIPs) nostr protocol github](https://github.com/nostr-protocol/nips#readme) has detailed descriptions of technical implementations and is very well-documented.
 - [usenostr.org](https://usenostr.org/) for a basic and approachable description
 - [awesome-nostr](https://github.com/aljazceru/awesome-nostr#readme) for a complete list of tools and resources.

In short, nostr is a communication protocol that provides a relatively simple framework for events and metadata to be published to servers (relays) after being cryptographically signed using a private key. In orther words, the data published to the relays is visible by anyone, but provably created by the person who holds the private key associated with the event. A technical description of how the events are created can be found [here](https://github.com/nostr-protocol/nips/blob/master/01.md#events-and-signatures) and a current list of the different event types [here](https://github.com/nostr-protocol/nips#event-kinds).

Because of the simplicity and openness of the nostr protocol a long list of tools and clients with different purposes have already been built on the nostr protocol as can be seen on [awesome-nostr](https://github.com/aljazceru/awesome-nostr#readme). There will continue to be more as new people are exposed to the protocol and the accepted event types continue to expand.

Fundamentally, nostr relays and clients are built on a relatively simple set of tools - most of which can be described nicely by looking at the modules of `python-nostr`, which is what we'll do here. Specifically, covered topics in this notebook can be accessed quickly via the sidebar and include:

 - `PrivateKey` and `PublicKey`
 - `Event`
 - `Relay`, `RelayManager`, and the `MessagePool`
 - `Subscription`

## Private and Public Keys
In nostr, the private key is the root of "account" ownership. The public key is derived from a private key using the same eliptic curve cryptography that is used for bitcoint private and public key pairs. nostr events are then signed using [Schnorr Signatures](https://bips.xyz/340), which prove "ownership" of the event. We will cover events in more detail later.

In this documentation you will see two versions of the `PublicKey` and `PrivateKey` classes, one version loaded directly from the `python-nostr.key` module as `key.Class` and one with added hex and bech32 functionality as denoted without `key.` in front.

First, let's look at the `PrivateKey` classes

In [ ]:
#| export
from nostr import key
from nostr import bech32
import secp256k1
from fastcore.utils import patch

In [ ]:
#| export
#| hide

class PrivateKey(key.PrivateKey):
    """a class to manage private keys inherited from
    python-nostr.key.PrivateKey, with a from_hex() class method added
    """
    def __init__(self, *args, **kwargs):
        """create a private key from raw bytes
        """
        super().__init__(*args, **kwargs)
        sk = secp256k1.PrivateKey(self.raw_secret)
        self.public_key = PublicKey(sk.pubkey.serialize()[1:])

    def __repr__(self):
        pubkey = self.public_key.bech32()
        return f'PrivateKey({pubkey[:10]}...{pubkey[-10:]})'

    @classmethod
    def from_hex(cls, hex: str) -> 'PrivateKey':
        return cls(bytes.fromhex(hex))

class PublicKey(key.PublicKey):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def __repr__(self):
        pubkey = self.bech32()
        return f'PublicKey({pubkey[:10]}...{pubkey[-10:]})'
    
    @classmethod
    def from_npub(cls, npub: str):
        """ Load a PublicKey from its bech32/npub form """
        hrp, data, spec = bech32.bech32_decode(npub)
        raw_bytes = bech32.convertbits(data, 5, 8)[:-1]
        return cls(bytes(raw_bytes))

    @classmethod
    def from_hex(cls, hex: str) -> 'PrivateKey':
        return cls(bytes.fromhex(hex))

By default the class is instantiated with raw bytes as shown below

In [ ]:
show_doc(key.PrivateKey)

---

### PrivateKey

>      PrivateKey (raw_secret:bytes=None)

Initialize self.  See help(type(self)) for accurate signature.

Instantiation with no argument gives a new random private key

In [ ]:
private_key = PrivateKey()
print(private_key)

PrivateKey(npub1frtp4...tu7s4ewqxm)


more commonly you will likely instantiate the private key from a hex string

In [ ]:
show_doc(PrivateKey.from_hex)

---

[source](https://github.com/armstrys/nostrfastr/blob/main/nostrfastr/nostr.py#L29){target="_blank" style="float:right; font-size:smaller"}

### PrivateKey.from_hex

>      PrivateKey.from_hex (hex:str)

In [ ]:
private_key_hex = private_key.hex()
the_same_private_key = PrivateKey.from_hex(private_key_hex)
assert private_key.hex() == the_same_private_key.hex()

or after the introduction of [bech32 entities in NIP-19](https://github.com/nostr-protocol/nips/blob/master/19.md) (private keys denoted as `nsec...`) like so

In [ ]:
show_doc(PrivateKey.from_nsec)

---

### PrivateKey.from_nsec

>      PrivateKey.from_nsec (nsec:str)

Load a PrivateKey from its bech32/nsec form

In [ ]:
private_key_bech32 = private_key.bech32()
the_same_private_key = PrivateKey.from_nsec(private_key_bech32)
assert private_key.bech32() == the_same_private_key.bech32()

The `PublicKey` class from `python-nostr` has very similar characteristics to the private key class, but with the key difference that the public key can be generated from the private key and the reverse is not possible by design. Below are the docs for instantiating a `PublicKey` object in `python-nostr`

In [ ]:
show_doc(key.PublicKey)

---

### PublicKey

>      PublicKey (raw_bytes:bytes)

Initialize self.  See help(type(self)) for accurate signature.

Unlike the `PrivateKey` class, the `PublicKey` class requires an input to be created. A random public key would mean that no one knows the associated private key and so that is not allowed. Instead, we will get our public key from the private key we already generated

In [ ]:
public_key = private_key.public_key
print(public_key)
assert isinstance(public_key, PublicKey)

PublicKey(npub1frtp4...tu7s4ewqxm)


The public can can also be retrieved from a public hex

In [ ]:
show_doc(PublicKey.from_hex)

---

[source](https://github.com/armstrys/nostrfastr/blob/main/nostrfastr/nostr.py#L48){target="_blank" style="float:right; font-size:smaller"}

### PublicKey.from_hex

>      PublicKey.from_hex (hex:str)

In [ ]:
public_key_hex = public_key.hex()
the_same_public_key = PublicKey.from_hex(public_key_hex)
assert public_key.hex() == the_same_public_key.hex()

or an npub bech32 encoded public key

In [ ]:
show_doc(PublicKey.from_npub)

---

[source](https://github.com/armstrys/nostrfastr/blob/main/nostrfastr/nostr.py#L41){target="_blank" style="float:right; font-size:smaller"}

### PublicKey.from_npub

>      PublicKey.from_npub (npub:str)

Load a PublicKey from its bech32/npub form

In [ ]:
public_key_bech32 = public_key.bech32()
the_same_public_key = PublicKey.from_npub(public_key_bech32)
assert public_key.bech32() == the_same_public_key.bech32()

## Nostr Events
Events are the key substance through which user communication happens over Nostr. Right now, the most common event type is a `text_note`, but there are [many other event kinds](https://github.com/nostr-protocol/nips#event-kinds) that can be published to a nostr relay. Events are generally made up of the following:

 - `id` - the hash of the event content - if the content of the a created event changes on a relay the old `id` would no longer match the hash of the content
 - `pubkey` - the public key of the author of the event
 - `created_at` - a time stamp given to the event at time of creation
 - `kind` - integer labeling the kind of event
 - `tags` - a list of arrays with tags such as `e` for related events or `p` for related people (like mentions)
 - `content` - the content of an event - the format of the content will vary depending on the kind of event. For a `text_note` (kind 1) this would be a string
 - `signature` - a signature generated from the private key associated with the public key **and** the event ID field - a valid signature ensures that anyone can check that a specific event was in fact signed by the private key of the author and that the content has not changed from the time the event was signed.

 > **Note:** the above all applies specifically to event *creation*. An event can be republished across many relays by any party as long as the event data has not changed. Some relays have limits on the age of an event, but where the event lives and who put it there is completely irrelevant to the validity of an event. This has many interesting implications, some of which will be explored elsewhere in this module.

In `python-nostr` we create events like so:

In [ ]:
from nostr.event import Event, EventKind
import time
import pprint
from fastcore.test import test_fail

In [ ]:
show_doc(Event)

---

### Event

>      Event (public_key:str, content:str, created_at:int=1672932400,
>             kind:int=<EventKind.TEXT_NOTE: 1>, tags:list[list[str]]=[],
>             id:str=None, signature:str=None)

Initialize self.  See help(type(self)) for accurate signature.

As example, we can create a new event

In [ ]:
event = Event(
    public_key=public_key.hex(),
    content='this is a test',
    kind=EventKind.TEXT_NOTE,
    created_at=int(time.time(),)
)
pprint.pprint(event.to_json_object())

{'content': 'this is a test',
 'created_at': 1672932400,
 'id': 'efd544f058c763d90a166b9e2840e5d6e4e0960e2ec8acbd9b253fb35b153ecc',
 'kind': <EventKind.TEXT_NOTE: 1>,
 'pubkey': '48d61a92f14107043603490c44c5dafdb65feb6ec3a030657c7c322b50db5f3d',
 'sig': None,
 'tags': []}


The id hash is automatically generated from the event content, but the signature is blank because the event has not been signed with a private key. Below we can test signing an event with our real private key and also a random private key. We see that one event verifies that the public key, signature, and event id all match, while the event signed with the wrong private key does not verify.

In [ ]:
event.sign(PrivateKey().hex())
assert not event.verify()
event.sign(private_key.hex())
assert event.verify()
pprint.pprint(event.to_json_object())

{'content': 'this is a test',
 'created_at': 1672932400,
 'id': 'efd544f058c763d90a166b9e2840e5d6e4e0960e2ec8acbd9b253fb35b153ecc',
 'kind': <EventKind.TEXT_NOTE: 1>,
 'pubkey': '48d61a92f14107043603490c44c5dafdb65feb6ec3a030657c7c322b50db5f3d',
 'sig': 'bc8c7b50b509e332e08e417677d37449ae02e2a2844bd7283b1c7647b9ea51f1b77d6503ed95fdef14e47342f2be7e9715af14fb027ccceca02d7bb3a0b99a13',
 'tags': []}


Additionally, if we change the content and don't recompute both the `ID` and the `signature` then the event will not verify.

In [ ]:
event.created_at = time.time()
event.id = event.compute_id(public_key=event.public_key, created_at=event.created_at, kind=event.kind, tags=event.tags, content=event.content)
assert not event.verify()

event.created_at = time.time()
event.sign(private_key.hex())
assert not event.verify()

As long as we do both of these things the event verifies event with changed data.

In [ ]:
event.created_at = time.time()
event.id = event.compute_id(public_key=event.public_key, created_at=event.created_at, kind=event.kind, tags=event.tags, content=event.content)
event.sign(private_key.hex())
assert event.verify()

`python-nostr` also supports other event types. The basic client module in this library can help build the content and tags of other event types in the correct format.

In [ ]:
for kind in EventKind:
    display(kind)

<EventKind.SET_METADATA: 0>

<EventKind.TEXT_NOTE: 1>

<EventKind.RECOMMEND_RELAY: 2>

<EventKind.CONTACTS: 3>

<EventKind.ENCRYPTED_DIRECT_MESSAGE: 4>

<EventKind.DELETE: 5>

## Relays, Relay Management, and the Message Pool
Though this package is primarily focused on the tools needed to run a nostr client - we will talk a bit about only enough to understand how they serve the client. More detailed information on relays can always be found in the [NIPs section of the nostr github repository](https://github.com/nostr-protocol/nips). For testing in this package we are going to use a python package called `nostr-relay`, which is installed alongside this package. It will run a local server that we can connect to instead of pinging public servers over and over with tests.

Relays act as a simple data server for clients to interact with. Clients can publish events a relay and make requests (in the form of subscriptions) to get events back from a relay.

Connections between clients and relays are made over a 'websocket' (typically handled by [websockets](https://websockets.readthedocs.io/en/stable/) in Python). A single websocket between a client and a relay can handle all data requests and typically should remain open as long as the client is live. Repeated connection and disconnections of websockets can put extra, unneeded strain on relays.

The message types that can be exchanged between a client and a relay are outlined [here](https://github.com/nostr-protocol/nips#message-types):

 - client to relay
     - `EVENT` - A nostr event being published to a relay
     - `REQ` - a client requesting a subscription to events from a relay
     - `CLOSE` - a client closing a subscription
 - relay to client
     - `EVENT` - an incoming event from a relay
     - `NOTICE` - human-readable text with an explanation of any issues
     - `EOSE` - a notice that all past stored messages have been transferred and that all future messages are newly published
     - `OK` - a True/False response if an EVENT was published successfully

`python-nostr` uses a `MessagePool` class to `Queue` the incoming stream of messages from the individial `Relay` websockets that are all running on different threads.

This package implements slightly modified versions of the `Relay`, `RelayManager`, and `MessagePool` classes found in `python-nostr` the basic usage is nearly identical, with a few added features that make building a the `Client` class easier. Added features that aren't yet included in `python-nostr`:

 - a `Connection` context manager that is used to open connections on a `Relay` or `RelayManager` object using a `with` statement
 - `Relay`
     - an `open_connections` method on a single relay to mirror the thread-based connection opening used in the `RelayManager` class.
     - an `is_connected` property to see if the websocket is open
 - `RelayManager`
     - additional connection management methods to make relay manager more robust to inactive connections
     - an `__iter__` method that allows the client to access the `Relay` objects by iterating directly on the `RelayManager` object instead of `RelayManager.relays.values()`

In [ ]:
#| hide

import os

In [ ]:
#| hide
os.system('nostr-relay -c ../nostr-relay/nostr-relay-config.yml serve >/dev/null 2>&1 &')
time.sleep(5)

In [ ]:
#| hide
#| export
import json
import time
import warnings
import threading
from threading import Lock
from typing import Union
from queue import Queue
from nostr import message_pool
from nostr import relay, relay_manager
from nostr.relay import RelayPolicy
from nostr.message_pool import EventMessage, NoticeMessage, EndOfStoredEventsMessage
from nostr.message_type import RelayMessageType
from nostr.event import Event

In [ ]:
from nostr import relay, relay_manager

In [ ]:
#| hide
#| export

class MessagePool(relay_manager.MessagePool):
    def __init__(self, first_response_only: bool = True):
        self.first_response_only = first_response_only
        self.events: Queue[EventMessage] = Queue()
        self.notices: Queue[NoticeMessage] = Queue()
        self.eose_notices: Queue[EndOfStoredEventsMessage] = Queue()
        self._unique_objects: set = set()
        self.lock: Lock = Lock()

    def _process_message(self, message: str, url: str):
        message_json = json.loads(message)
        message_type = message_json[0]
        if message_type == RelayMessageType.EVENT:
            subscription_id = message_json[1]
            e = message_json[2]
            event = Event(e['pubkey'], e['content'], e['created_at'], e['kind'], e['tags'], e['id'], e['sig'])
            with self.lock:
                if self.first_response_only:
                    object_id = event.id
                else:
                    object_id = f'{event.id}:{url}'
                if object_id not in self._unique_objects:
                    self.events.put(EventMessage(event, subscription_id, url))
                    self._unique_objects.add(event.id)


In [ ]:
#| hide
#| export

class Connection:
    def __init__(self, relay_or_manager: Union[relay.Relay, relay_manager.RelayManager],
                 *args, **kwargs):
        self.relay_manager = relay_or_manager
        self.conn = self.relay_manager.open_connections(*args, **kwargs)
    def __enter__(self):
        return self.conn
    def __exit__(self, ex_type, ex_value, traceback):
        self.relay_manager.close_connections()
        return False


class Relay(relay.Relay):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def __repr__(self):
        return json.dumps(self.to_json_object(), indent=2)

    @property
    def is_connected(self) -> bool:
        return False if self.ws.sock is None else self.ws.sock.connected
    
    def open_connections(self, ssl_options: dict={}):
        threading.Thread(
                target=self.connect,
                args=(ssl_options,),
                name=f"{self.url}-thread"
        ).start()
    
    def close(self):
        if self.ws.sock is not None:
            self.ws.close()
    
    def close_connections(self):
        self.close()
    
    def connection(self, *args, **kwargs):
        return Connection(self, *args, **kwargs)


In [ ]:
#| hide
#| export

class RelayManager(relay_manager.RelayManager):
    def __init__(self, first_response_only: bool = True,  *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.relays: dict[str, Relay] = {}
        self.message_pool = MessagePool(first_response_only=first_response_only)
        self._is_connected = False

    def __iter__(self):
        return iter(self.relays.values())
    
    def connection(self, *args, **kwargs):
        return Connection(self, *args, **kwargs)
    
    def open_connections(self, ssl_options: dict=None):
        for relay in self.relays.values():
            if not relay.is_connected:
                threading.Thread(
                    target=relay.connect,
                    args=(ssl_options,),
                    name=f"{relay.url}-thread"
                ).start()
        time.sleep(2)
        self.remove_closed_relays()
        assert all(self.connection_statuses.values())
        self._is_connected = True
    
    def close_connections(self):
        for relay in self.relays.values():
            relay.close()

        assert not any(self.connection_statuses.values())
        self._is_connected = False

    def remove_closed_relays(self):
        for url, connected in self.connection_statuses.items():
            if not connected:
                warnings.warn(
                    f'{url} is not connected... removing relay.'
                )
                self.remove_relay(url=url)

    def add_relay(self, url: str, read: bool=True, write: bool=True, subscriptions={}):
        policy = RelayPolicy(read, write)
        relay = Relay(url, policy, self.message_pool, subscriptions)
        self.relays[url] = relay
    
    def remove_relay(self, url: str):
        self.relays[url].close()
        self.relays.pop(url)

    @property
    def connection_statuses(self) -> dict:
        """gets the url and connection statuses of relays

        Returns:
            dict: bool of connection statuses
        """
        statuses = [relay.is_connected for relay in self]
        return dict(zip(self.relays.keys(), statuses))

### Relay
The documentation below shows how a Relay is instantiated using `python-nostr`. The relay policy is defined by the `RelayPolicy` class

In [ ]:
show_doc(relay.Relay)

---

### Relay

>      Relay (url:str, policy:nostr.relay.RelayPolicy,
>             message_pool:nostr.message_pool.MessagePool,
>             subscriptions:dict[str,nostr.subscription.Subscription]={})

Initialize self.  See help(type(self)) for accurate signature.

Remember that in this package we will use a `Relay` class that inherits the `python-nostr` relay class. The same will go for the relay manager and the message pool.

In [ ]:
assert issubclass(Relay, relay.Relay) and \
       issubclass(RelayManager, relay_manager.RelayManager) and \
       issubclass(MessagePool, message_pool.MessagePool)

Below we test using the context manager from the `Relay` class in this package to connect and disconnect to a relay.

In [ ]:
import ssl

In [ ]:
url='ws://127.0.0.1:6969'

a_relay = Relay(
    url=url,
    policy=RelayPolicy(),
    message_pool=MessagePool()
    )

assert not a_relay.is_connected

with a_relay.connection(ssl_options={'cert_reqs': ssl.CERT_NONE}):
    time.sleep(.5)
    print(f'Is the relay connected? {a_relay.is_connected}')
assert not a_relay.is_connected
time.sleep(.5)

Is the relay connected? False


and make sure that when it errors out it properly reports the error and proceeds to safely close the connection to the relay

In [ ]:
from fastcore.test import test_fail

In [ ]:
def error_in_context():
    with a_relay.connection(ssl_options={'cert_reqs': ssl.CERT_NONE}):
        time.sleep(.5)
        raise Error()
test_fail(error_in_context)
assert not a_relay.is_connected

Let's try to publish an event to a our local relay - typically you would do this through the relay manager as to publish content across multiple relays, but for testing purposes we will only use one.

In [ ]:
show_doc(Relay.publish)

---

### Relay.publish

>      Relay.publish (message:str)

In [ ]:
from nostr import message_type

We need to get our message to the relay [in the right format as specified by NIP-01](https://github.com/nostr-protocol/nips/blob/master/01.md#from-client-to-relay-sending-events-and-creating-subscriptions)

In [ ]:
message = [message_type.ClientMessageType.EVENT, event.to_json_object()]
message = json.dumps(message)
print(message)

["EVENT", {"id": "063ef95ee423321621174205df07015baffbb792cfdab9112fa86549304d1ff9", "pubkey": "48d61a92f14107043603490c44c5dafdb65feb6ec3a030657c7c322b50db5f3d", "created_at": 1672932401.5833979, "kind": 1, "tags": [], "content": "this is a test", "sig": "350f831f2b28ffbc2fe65468f3a090eecf03a4c90a818d2d286eeeddc6d495a8984c82b9bf51827903e6c15f55b24ea60c7a57113b107e0a35bfcc48c3d48401"}]


And now we can publish it.

In [ ]:
with a_relay.connection(ssl_options={'cert_reqs': ssl.CERT_NONE}):
    time.sleep(.5)
    a_relay.publish(message)

We will try to read this later.

### Relay Manager
Generally, most interactions will happen through the `RelayManager` class. The relay manager allows easy interaction across multiple relays. Key methods are documented below

In [ ]:
show_doc(RelayManager.add_relay)

---

[source](https://github.com/armstrys/nostrfastr/blob/main/nostrfastr/nostr.py#L177){target="_blank" style="float:right; font-size:smaller"}

### RelayManager.add_relay

>      RelayManager.add_relay (url:str, read:bool=True, write:bool=True,
>                              subscriptions={})

In [ ]:
show_doc(RelayManager.remove_relay)

---

[source](https://github.com/armstrys/nostrfastr/blob/main/nostrfastr/nostr.py#L182){target="_blank" style="float:right; font-size:smaller"}

### RelayManager.remove_relay

>      RelayManager.remove_relay (url:str)

In [ ]:
show_doc(RelayManager.publish_message)

---

### RelayManager.publish_message

>      RelayManager.publish_message (message:str)

You can also access connection statuses of the relays with `RelayManager.connection_statuses`. Below we use that property to check connections after making a `RelayManager` object, adding a `Relay` and connecting with the context manager.

In [ ]:
manager = RelayManager()
urls=[url]
for url in urls:
    manager.add_relay(url=url)

with manager.connection():
    print('opening')
    print(manager.connection_statuses)
    assert all(manager.connection_statuses.values())
    print('closing')
print(manager.connection_statuses)
assert not any(manager.connection_statuses.values())


opening
{'ws://127.0.0.1:6969': True}
closing
{'ws://127.0.0.1:6969': False}


`Relay` objects that don't connect will be remove automatically and throw a warning.

In [ ]:
manager = RelayManager()
urls=['ws://this-relay-doesnt-exist.com']
for u in urls:
    manager.add_relay(url=u)

with manager.connection():
    pass
    assert len(manager.relays) == 0

/var/folders/3x/w083pw853418fqm2blj59w480000gn/T/ipykernel_62107/3905095593.py:40: UserWarning: ws://this-relay-doesnt-exist.com is not connected... removing relay.
  warnings.warn(


### Message Pool

Because we have multiple open websockets, the connections are threaded in Python. The `MessagePool` class acts as a shared Queue where different types of messages from the relay can all be accessed. The only significant change from the `python-nostr MessagePool` to this package is allowing the client to chose whether it should only keep the first event or whether it should keep multiples of event messages across all relays. Keeping all responses from all relays will allow the client to check how well distributed the user content is across the relays.

In [ ]:
show_doc(MessagePool)

---

[source](https://github.com/armstrys/nostrfastr/blob/main/nostrfastr/nostr.py#L67){target="_blank" style="float:right; font-size:smaller"}

### MessagePool

>      MessagePool (first_response_only:bool=True)

Initialize self.  See help(type(self)) for accurate signature.

The key events to access the `MessagePool` in `python-nostr` are listed below. We will add more functionality at the client level to modify how these events are handled upon receipt.

In [ ]:
show_doc(MessagePool.get_event)

---

### MessagePool.get_event

>      MessagePool.get_event ()

In [ ]:
show_doc(MessagePool.get_notice)

---

### MessagePool.get_notice

>      MessagePool.get_notice ()

In [ ]:
show_doc(MessagePool.get_eose_notice)

---

### MessagePool.get_eose_notice

>      MessagePool.get_eose_notice ()

In order to populate the message pool we need subscriptions.

## Subscriptions
The nostr protocol requires clients to make requests (as subscriptions) to the relays in order to get events sent back (in our case, into the `MessagePool`). The `REQ` (request or subscription) query is also clearly outlined in [NIP-01](https://github.com/nostr-protocol/nips/blob/master/01.md#from-client-to-relay-sending-events-and-creating-subscriptions). A request must be labeled with a `subscription_id` and a series of filters that the relay will use to determine which events it should return. Current options for a single filter are as follow:

- `ids` - a list of event IDs
- `authors` - a list of authors (pubkeys), which could be something like a contact list
- `kinds` - a list of kinds, such as `text_notes` (1) or `set_metadata` (0)
- `tags` - a list of tags, like `e` event or `p` people tags
- `since` - a time stamp that defines the earliest `created_at` that should be sent 
- `until` - a time stamp that defines the latest `created_at` that should be sent 
- `limit` - a limit of events to return upon initial request to keep the client from being overwhelmed if the query is wide

A list of filters is provided in a `REQ` message and the following happens:

- the relay returns all events in past storage that meet the union of all the filter specifications (i.e. each filter is handled separately)
- the relay would then send an `End Of Stored Events` notice with the `subscription_id`
- the relay will continue to send newly published events that meet the criteria until 1 of the following occurs:
     - the websocket is closed (by any measure - intentional closure or client crash)
     - no more events are being published because the `until` clause of a subscription is met
     - the client sends a request with the same `subscription_id` and it overwrites the initial behavior
     - the client sends a `CLOSE` message to close the `subscription_id`

In [ ]:
from nostr import filter
from nostr import subscription
import uuid

In [ ]:
show_doc(filter.Filter)

---

### Filter

>      Filter (ids:list[str]=None, kinds:list[int]=None, authors:list[str]=None,
>              since:int=None, until:int=None, tags:dict[str,list[str]]=None,
>              limit:int=None)

Initialize self.  See help(type(self)) for accurate signature.

In [ ]:
show_doc(subscription.Subscription)

---

### Subscription

>      Subscription (id:str, filters:nostr.filter.Filters=None)

Initialize self.  See help(type(self)) for accurate signature.

We can create a subscription to request the event we created earlier

In [ ]:
filters = filter.Filters([filter.Filter(ids=[event.id])])
subscription_id = str(uuid.uuid4())
request = [message_type.ClientMessageType.REQUEST, subscription_id]
request.extend(filters.to_json_array())
message = json.dumps(request)
print(message)

["REQ", "9e9fead8-2b3d-4040-9999-c79ecf58def7", {"ids": ["063ef95ee423321621174205df07015baffbb792cfdab9112fa86549304d1ff9"]}]


Now we can get that from the relay we published it to

In [ ]:
manager = RelayManager()
manager.add_relay(url)

with manager.connection(ssl_options={'cert_reqs': ssl.CERT_NONE}):
    time.sleep(.5)
    manager.add_subscription(subscription_id, filters)
    manager.publish_message(message)
    time.sleep(.5)
    while manager.message_pool.has_events():
        event_msg = manager.message_pool.get_event()
        print(f'event received from {event_msg.url} '
                f'with subscription id {event_msg.url}\n\t'
                f'{event_msg.event.content}')

event received from ws://127.0.0.1:6969 with subscription id ws://127.0.0.1:6969
	this is a test


We will use the `Client` class in this package to make subscriptions and publishing events even easier.

In [ ]:
#| hide
# os.system('pkill pkill gunicorn:\\')

0

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()